In [127]:
import pandas as pd
import numpy as np
import os
from data.contracts import contract_addresses

transfers_cols = ['DateTime','From','To','Quantity','Method']

## aUSDC Aave interest bearing USDC
# Holders, filter out contracts
aUSDC_holders = pd.read_csv('data/aUSDC/aUSDC_holders.csv').sort_values(by='Balance', ascending=False)
aUSDC_holders = aUSDC_holders[~aUSDC_holders['HolderAddress'].isin(contract_addresses)].drop([1259]).set_index('HolderAddress')
aUSDC_holders = pd.cut(aUSDC_holders['Balance'], bins=[0,300,3000,30000,300000,3000000,30000000,300000000], labels=[0,1,2,3,4,5,6]).to_frame().rename(columns={'Balance': 'balance'})


# Transfers, filter out contracts, keep only transactions that mint/burn aUSDC, convert qty to float
aUSDC_transfers = pd.DataFrame()
for f in os.listdir('data/aUSDC/'):
    if f.endswith('.csv') and (f[0] in '0123'):
        data = pd.read_csv('data/aUSDC/'+f, usecols=transfers_cols)
        if data.loc[0]['DateTime'].startswith('2021'):
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
        else:
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')
        if len(aUSDC_transfers):
            aUSDC_transfers = pd.concat([aUSDC_transfers,data])
        else:
            aUSDC_transfers = data
            
flash_loan_addresses = list(set(list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['From'].unique()) + list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['To'].unique())))
flash_loan_addresses.remove('0x0000000000000000000000000000000000000000')

aUSDC_transfers = aUSDC_transfers[(~aUSDC_transfers['From'].isin(contract_addresses))&(~aUSDC_transfers['To'].isin(contract_addresses))]
aUSDC_transfers = aUSDC_transfers[(aUSDC_transfers['From']=='0x0000000000000000000000000000000000000000')|(aUSDC_transfers['To']=='0x0000000000000000000000000000000000000000')].reset_index(drop=True)
aUSDC_transfers['Quantity'] = aUSDC_transfers['Quantity'].apply(lambda x: float(''.join(x.split(','))))

# Agg
from__ = aUSDC_transfers[aUSDC_transfers['From']!='0x0000000000000000000000000000000000000000'].rename(columns={"From": "Address"}).drop(columns='To')
to__ = aUSDC_transfers[aUSDC_transfers['To']!='0x0000000000000000000000000000000000000000'].rename(columns={"To": "Address"}).drop(columns='From')
combined_aUSDC_transfers = pd.concat([from__, to__])
latest_day = combined_aUSDC_transfers['DateTime'].max()
# 30-day
combined_aUSDC_transfers_30 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=30))]
num_transactions_30 = np.log10(combined_aUSDC_transfers_30['Address'].value_counts()).to_frame().rename(columns={'Address': '30_day_log10_num_transactions'})
# 90-day
combined_aUSDC_transfers_90 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=90))]
num_transactions_90 = np.log10(combined_aUSDC_transfers_90['Address'].value_counts()).to_frame().rename(columns={'Address': '90_day_log10_num_transactions'})
# 180-day
combined_aUSDC_transfers_180 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=180))]
num_transactions_180 = np.log10(combined_aUSDC_transfers_180['Address'].value_counts()).to_frame().rename(columns={'Address': '180_day_log10_num_transactions'})


# Feature engineering
to_ = pd.DataFrame(aUSDC_transfers['To'].value_counts())
from_ = pd.DataFrame(aUSDC_transfers['From'].value_counts())
features = to_.join(from_, how='outer').rename(columns={"To": "num_USDC_deposits", "From": "num_USDC_withdrawals"}).fillna(0)
features['flash_loan'] = [True if i in flash_loan_addresses else False for i in features.index]
features = features.join([aUSDC_holders,num_transactions_30,num_transactions_90,num_transactions_180], how='left').fillna(0)

In [130]:
features

,num_USDC_deposits,num_USDC_withdrawals,flash_loan,balance,30_day_log10_num_transactions,90_day_log10_num_transactions,180_day_log10_num_transactions
0x0000000000000000000000000000000000000000,14728.0,18392.0,False,0,0.00000,0.00000,0.00000
0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2.0,2.0,False,0,0.00000,0.60206,0.60206
0x000000003ce0cf2c037493b1dc087204bd7f713e,1.0,0.0,False,0,0.00000,0.00000,0.00000
0x0000000484f2217f1a64eb6d24b5cee446faeae5,1.0,1.0,False,0,0.00000,0.00000,0.30103
0x000000aaee6a496aaf7b7452518781786313400f,0.0,4.0,False,0,0.00000,0.00000,0.60206
...,...,...,...,...,...,...,...
0xffda397f3de16ec7dad3553c220a3b2b292e0255,0.0,1.0,False,0,0.00000,0.00000,0.00000
0xffdb8f98cfac992824dfac77c88d2107c6df3ab5,1.0,1.0,False,0,0.30103,0.30103,0.30103
0xffeb122a31cd9bba428adcc1427c33dfca3aeb10,1.0,1.0,False,0,0.00000,0.00000,0.30103
0xffefdcfff613c9bbb9928f6ff44f07c7b562bfdf,1.0,1.0,False,0,0.00000,0.30103,0.30103


In [ ]:
average transaction size
frequency
time series data

more tokens

MORE HISTORICAL DATA